# <center> MATH&ML-6. Математический анализ в контексте задачи оптимизации. Часть III
---

### 2. Градиентный спуск: применение и модификации

In [18]:
# Задание 2.7
# Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии
# Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# Загрузите стандартный датасет об алмазах из библиотеки Seaborn:
import seaborn as sns
df = sns.load_dataset('diamonds')
# Удалите часть признаков:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
# Закодируйте категориальные признаки:
df = pd.get_dummies(df, drop_first=True)
# Логарифмируйте признаки:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])
# Определите целевую переменную и предикторы:
X = df.drop(columns="price")
y = df["price"]
# Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor)
reg = SGDRegressor()
# Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:
grid = {
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 10),
    "l1_ratio": np.linspace(0, 1, 10),
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)
    }

grid_search = GridSearchCV(
    estimator=reg,
    param_grid=grid,
    n_jobs=-1
    )
grid_search.fit(X_train, y_train)
y_test_pred = grid_search.predict(X_test)
round(metrics.mean_squared_error(y_test, y_test_pred), 3)

0.047

---
### 3. Метод Ньютона

In [19]:
x = 0.6296335078534031
f = 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
dx = 30*x**4 - 20*x**3 - 12*x**2 + 6*x
x - f / dx

0.6286680781673306

In [20]:
# Задание 3.1
def newton(x):
    def f(x):
        return 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
    def dx(x):
        return 30*x**4 - 20*x**3 - 12*x**2 + 6*x
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.001:
            print(round(x_new, 3))
            break
        x = x_new

newton(0.7)

0.629


In [21]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

In [22]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

#21.695121951219512
#11.734125501243229
#7.1123493600499685
#5.365000391507974
#5.015260627016227
#5.000029000201801
#5.000000000105126
#5.0

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


In [23]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)
# 5

5.0

In [24]:
from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)
# 5

5.0

In [25]:
# Задание 3.6
def f(x):
    return x**3 - 72*x - 220

def dx(x):
    return 3*x**2 - 72

x0 = 12

def newton(f, dx, x):
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.001:
            print(round(x_new, 3))
            break
        x = x_new

newton(f, dx, x0)

9.727


In [27]:
# Задание 3.7
def f(x):
    return x**2 + 9*x - 5

def dx(x):
    return 2*x + 9

x0 = 2.2

def newton(f, dx, x):
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.01:
            print(round(x_new, 2))
            break
        x = x_new

newton(f, dx, x0)

0.52


In [32]:
# Задание 3.9
def dx(x):
    return 24*x**2 - 4*x

def ddx(x):
    return 48*x -4

x0 = 42

def newton(f, dx, x):
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.0001:
            print(round(x_new, 3))
            break
        x = x_new

newton(dx, ddx, x0)

0.167


---
### 4. Квазиньютоновские методы

Подгрузим необходимые библиотеки:

In [33]:
import numpy as np
from scipy.optimize import minimize

Определим функцию, которую будем оптимизировать. Вместо отдельных $x$ и $y$ можно взять координаты единого вектора:

In [34]:
def func(x):
    return x[0]**2.0 + x[1]**2.0

Теперь определим градиент для функции:

In [35]:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

Зададим начальную точку:

In [36]:
x_0 = [1.0, 1.0]

Определим алгоритм:

In [37]:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

Выведем результаты:

In [38]:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
 
#Статус оптимизации Optimization terminated successfully.
#Количество оценок: 3
#Решение: f([0. 0.]) = 0.00000

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Итак, мы получили, что минимум функции достигается в точке $(0,0)$. Значение функции в этой точке также равно нулю.

Можно повторить то же самое с вариацией  *L-BFGS-B*:

In [39]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Результат будет тем же самым.

In [41]:
# Задание 4.1
def func(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

def grad_func(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

x0 = [-400, -400]

result = minimize(func, x0, method='L-BFGS-B', jac=grad_func)
result

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -0.9999999999999218
        x: [-4.000e+00  1.000e+00]
      nit: 4
      jac: [ 2.900e-07  2.724e-07]
     nfev: 9
     njev: 9
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [42]:
# Задание 4.4
def f(x):
    return x**2 - 3*x + 45

def dx(x):
    return 2*x - 3

x0 = 10

result = minimize(f, x0, method='BFGS', jac=dx)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 42.75
        x: [ 1.500e+00]
      nit: 4
      jac: [ 0.000e+00]
 hess_inv: [[ 5.000e-01]]
     nfev: 5
     njev: 5

In [43]:
# Задание 4.5
minimize(f, x0, method='L-BFGS-B', jac=dx)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 42.75
        x: [ 1.500e+00]
      nit: 2
      jac: [ 0.000e+00]
     nfev: 3
     njev: 3
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>

In [60]:
# Задание 4.7
def f(dot):
    x = dot[0]
    y = dot[1]
    return x**4 + 6*y**2 + 10

def grad_f(dot):
    x = dot[0]
    y = dot[1]
    return np.array([4*x**3, 12*y])

x0 = [100, 100]

result_1 = minimize(f, x0, method='BFGS', jac=grad_f)
result_2 = minimize(f, x0, method='L-BFGS-B', jac=grad_f)
print(f'BFGS: \n {result_1} \n\n\nL-BFGS-B:  \n {result_2}')

BFGS: 
   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 10.000000030008898
        x: [ 1.316e-02  6.653e-14]
      nit: 34
      jac: [ 9.120e-06  7.984e-13]
 hess_inv: [[ 2.016e+02 -4.163e-09]
            [-4.163e-09  7.317e-02]]
     nfev: 37
     njev: 37 


L-BFGS-B:  
   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 10.00000000827103
        x: [-9.527e-03 -2.322e-06]
      nit: 37
      jac: [-3.459e-06 -2.786e-05]
     nfev: 40
     njev: 40
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


---
### 5. Линейное программирование